In [6]:

from bs4 import BeautifulSoup
import re
import time



In [7]:
###chromedriver must be first downloaded from here
###http://chromedriver.storage.googleapis.com/index.html?path=2.43/

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
### point the webdriver to the location where you downloaded the driver
browser = webdriver.Chrome("/home/javiersevillano/Descargas/webdrivers/chromedriver")
type(browser)
url="https://www.idealista.com/"
browser.get(url)

In [8]:
#give time to load the page
time.sleep(3)

In [9]:
##lets find and access the login page
id="your-account"  ### element for the log in
##browser.find_element_by_id(id)
try:
    login = browser.find_element_by_id(id)
    print('Found <%s> element with that id name!' % (login.text))
except:
    print('Was not able to find an element with that name.')
login.click()

Found <Acceso usuarios> element with that id name!


In [ ]:
###lets find and fill the login credentials
my_email= input('user email')
id="email"
email_elem = browser.find_element_by_id(id)
email_elem.tag_name
email_elem.clear()
email_elem.send_keys(my_email)

In [ ]:
pwd=input('user pwd')
id="pwd"
pwd_elem = browser.find_element_by_name(id)
pwd_elem.tag_name
pwd_elem.clear()
pwd_elem.send_keys(pwd)

In [ ]:
### press the login button
##lets find and access the login page

id="doLogin"  ### element for the log in
##browser.find_element_by_id(id)
try:
    login = browser.find_element_by_id(id)
    print('Found <%s> element with that id name!' % (login.text))
except:
    print('Was not able to find an element with that name.')
login.click()

In [10]:
#### press the button in favourires
favoritos=browser.find_element_by_xpath("//a[contains(@href, 'favoritos')]")
print(favoritos.text)
favoritos.click()

Tus favoritos


In [11]:
res=browser.page_source
soup=BeautifulSoup(res, "lxml")
print(soup.prettify()[:200])

<!DOCTYPE html>
<!--[if IE 8]><html class="ie8" lang="es" env="es" username="Sara Serrano Moreno"><![endif]-->
<!--[if IE 9]><html class="ie9" lang="es" env="es" username="Sara Serrano Moreno"><![endi


In [12]:
with open("/home/javiersevillano/Documentos/Testing Jup Lab/idealista.txt",'w') as f:
    f.write(soup.prettify())

In [13]:
def extract_flat(flat):
    my_flat={}
    #check if it is actually a flat
    if len(flat.select(".logo-container"))>0:
        return None
    else:
        ## gets the title and the href
        fav_icon=flat.select(".icon-fav")[0]
        my_flat['href']=fav_icon.get('href')
        my_flat['inmueble']=fav_icon.get('href')
        my_flat['inmueble']=re.search('(?<=inmueble/)\d+',my_flat['inmueble']).group(0)
        my_flat['href_comp']="https://www.idealista.com"+my_flat['href']
        my_flat['title']=fav_icon.get('title')
        
        ## gets the price row
        prices=flat.select(".price-row")
        my_flat['price']=prices[0].select(".item-price")[0].getText()
        
        other_info_price=[i.getText() for i in prices]
        my_flat['ext_price']=" ".join(other_info_price)
        
        ## gets the details
        details=[i.getText() for i in flat.select(".item-detail")]
        my_flat['detail']=" ".join(details)
        
        #isolating details
        for i in details:
            if "hab." in i: #search habitaciones
                my_flat['habs']=re.search("\d+",i).group(0)
            if "m"+u"\u00B2" in i: #search metros2 
                my_flat['metros']=re.search("\d+",i).group(0)
            if "planta" in i: #search metros2 
                my_flat['Edificio']=re.search(".+planta",i).group(0)
                my_flat['Edificio otros']=re.search("(?<=planta ).+",i).group(0)
        #webdriver.ActionChains(browser).key_down(Keys.CONTROL).send_keys('T').perform()
        url2=my_flat['href_comp']
        #browser.execute_script("window.open('https://www.google.com');")
        #browser.execute_script("window.open('"+url2+"')")
        browser.get(url2)
        flat_soup=BeautifulSoup(browser.page_source, "lxml")
        items=flat_soup.select(".details-property_features > ul > li")
        otros=[i.getText() for i in items]
        my_flat['otros detalles']="\n".join(otros)
            
        #print(x)
        time.sleep(0.5)
        browser.back()
        print(my_flat['inmueble'],"..............",my_flat['title'])
        return my_flat
    
        
favs=soup.select('.item-info-container')
#print(favs)

In [ ]:
extract_flat(favs[0])

In [14]:
all_flats=[]
for x in favs:
    all_flats.append(extract_flat(x))
    

82784921 .............. Piso en Acacias, Madrid
82792863 .............. Piso en calle de Las Naves, Madrid, Acacias, Madrid
82763185 .............. Piso en paseo de la Chopera, 9, Chopera, Madrid
82755932 .............. Piso en calle Labrador, 26, Acacias, Madrid
82742114 .............. Piso en calle de Las Naves, Madrid, 19, Acacias, Madrid
82752481 .............. Piso en paseo de Santa María de la Cabeza, Madrid, Chopera, Madrid
82751769 .............. Piso en plaza de las Peñuelas, Madrid, Acacias, Madrid
82293924 .............. Piso en paseo de la Chopera, Chopera, Madrid
82310775 .............. Piso en arganda, Acacias, Madrid
82607613 .............. Piso en Acacias, Madrid
82533442 .............. Piso en Acacias, Madrid
82720458 .............. Piso en Acacias, Madrid
82447908 .............. Piso en calle de Arganda, Acacias, Madrid
82647778 .............. Piso en calle de Toledo, 153, Acacias, Madrid
82647706 .............. Piso en paseo de las Yeserías, 31, Acacias, Madrid
82625

In [15]:
import pandas as pd

In [16]:
all_flats=[x for x in all_flats if x!=None]


In [18]:
df=pd.DataFrame(all_flats)
df.columns

Index(['Edificio', 'Edificio otros', 'detail', 'ext_price', 'habs', 'href',
       'href_comp', 'inmueble', 'metros', 'otros detalles', 'price', 'title'],
      dtype='object')

In [21]:
order=['inmueble','title','price','habs','metros','ext_price','detail','otros detalles', 'href_comp']
df=df[order]

In [22]:
df.to_csv('~/pisos_idealista.csv')


In [24]:

#extract_flat(favs[0])
#favs[0]

{'href': '/inmueble/82784921/', 'title': 'Piso en Acacias, Madrid', 'price': '580.000€', 'ext_price': ' 580.000€ Garaje incluido ', 'detail': '3 hab. 110 m² 2ª planta exterior con ascensor', 'habs': '3', 'metros': '110', 'planta': '2ª planta', 'planta otros': 'exterior con ascensor'}


<div class="item-info-container"> <div class="logo-branding"> <a href="/pro/engel-volkers-madrid/"> <div class="logo-border"></div> <img class=" show" data-ondemand-img="https://st3.idealista.com/20/d7/c3/engel-volkers-madrid.gif" src="https://st3.idealista.com/20/d7/c3/engel-volkers-madrid.gif"/> </a> </div> <a class="item-link icon-fav" href="/inmueble/82784921/" title="Piso en Acacias, Madrid"> Piso en Acacias, Madrid </a> <div class="row price-row clearfix"> <span class="item-price h2-simulated">580.000<span>€</span></span> <span>Garaje incluido</span> </div> <span class="item-detail">3 <small>hab.</small></span> <span class="item-detail">110 <small>m²</small></span> <span class="item-detail">2ª <small>planta exterior con ascensor</small></span> <div class="item-description description" style=""><p class="ellipsis" style="display: block;">Esta magnífica vivienda que Engel&amp;Völkers presenta en urbanización privada se encuentra ubicado en el barrio de Acacias. Al entrar en la vivi